In [1]:
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

coinURL = "https://beta.mcdgov.info/"
coinTableID = "polls-table"
proposalTableID = "sorted-table"

driver = webdriver.Chrome()
driver.get(coinURL)
elem = WebDriverWait(driver, 30).until(
EC.presence_of_element_located((By.ID, coinTableID)))
soup = BeautifulSoup(driver.page_source.encode('utf-8'))
driver.close()
table = soup.find(id = coinTableID)
rows=list()
for row in table.findAll("tr"):
    rows.append(row)
for row in rows:
    print("new")
    temp = []
    for td in row.find_all('td'):
        temp.append(td)
        #print(td)
        #print(td.prettify())
        if(len(temp)> 3):
            proposalURL = ("https://beta.mcdgov.info" + temp[0].find('a').get('href'))
            print("link: " + temp[0].find('a').string + " https://beta.mcdgov.info" + temp[0].find('a').get('href'))
            print("date: " + temp[1].string)
            print("name: " + temp[2].string)
            print("result: " + temp[3].string)
            try:
                driver = webdriver.Chrome()
                driver.get(proposalURL)
                elem = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.ID, proposalTableID)))
                soup2 = BeautifulSoup(driver.page_source.encode('utf-8'))
                print(soup2.prettify())
                proposalTable = soup2.find(id = proposalTableID)
                #print(table.prettify())
                driver.close()
                print( proposalTable.string)
                votes = list()
                for item in proposalTable.find_all('tr'):
                    votes.append(item)
                for vote in votes:
                    t2 =[]
                    for attr in vote.find_all('td'):
                        t2.append(attr)
                        print(t2)
                        if(len(t2) >= 5):
                            print("time: " + t2[0].string)
                            print("address: " + t2[1].string)
                            print("proxy " + t2[2].string)
                            print("operation: " + t2[3].string)
                            print("options: " + t2[4].string )
                            print("MKR: " + t2[5].string)
                    
            except Exception as err:            
                print("An exception occurred " + str(err))
            except KeyboardInterrupt:
                raise Exception("done")
            #table = soup.find(id = coinTableId)
        
        
#print(table)
# link, date, name, result


WebDriverException: Message: chrome not reachable
  (Session info: chrome=98.0.4758.102)
Stacktrace:
#0 0x55eeb0510b33 <unknown>
#1 0x55eeaffd956f <unknown>
#2 0x55eeaffc8184 <unknown>
#3 0x55eeaffc8b89 <unknown>
#4 0x55eeaffca8e2 <unknown>
#5 0x55eeaffc3076 <unknown>
#6 0x55eeaffdaad2 <unknown>
#7 0x55eeb003f9b1 <unknown>
#8 0x55eeb002cf53 <unknown>
#9 0x55eeb0002a0a <unknown>
#10 0x55eeb0003ad5 <unknown>
#11 0x55eeb05422fd <unknown>
#12 0x55eeb055b4bb <unknown>
#13 0x55eeb05440d5 <unknown>
#14 0x55eeb055c145 <unknown>
#15 0x55eeb0537aaf <unknown>
#16 0x55eeb0578ba8 <unknown>
#17 0x55eeb0578d28 <unknown>
#18 0x55eeb059348d <unknown>
#19 0x7f74f3e56609 <unknown>


In [110]:
from bs4 import BeautifulSoup
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
import os
import time

def getTable(url, waitID, ID):
    driver = webdriver.Chrome()
    driver.get(url)
    elem = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.ID, waitID)))
    soup = BeautifulSoup(driver.page_source.encode('utf-8'))
    #print(soup)
    driver.close()
    table = soup.find(id = ID)
    #print(table)
    proposals = list()
    for proposal in table.find_all("tr"):
        temp = []
        for item in proposal.find_all("td"):
            temp.append(item.string)
        proposals.append(temp)
    return proposals

def downloadCSV(url, waitId, downloadLocation):
    driver = webdriver.Chrome()
    driver.get(url)
    elem = WebDriverWait(driver, 300).until(
    EC.presence_of_element_located((By.ID, waitId)))
    l = driver.find_element_by_xpath("//button[@class='dt-button buttons-csv buttons-html5']")
    l.click()
    while not os.path.exists(downloadLocation):
        time.sleep(1)
    driver.close()

def getVotesFromCSV(filepath):
    votes = pd.read_csv(filepath)
    return votes

def printProps(proposals):
    for proposal in proposals:
        printProp(proposal)
            
def printProp(proposal):
     if(len(proposal) > 3):
        print("url: " + voteUrl + proposal[0])
        print("date:" + proposal[1])
        print("name:" + proposal[2])
        print("result:" + proposal[3])
        

def printVote(vote):
    if(len(vote) >= 5):
        print("time: " + vote[0])
        print("address: " + vote[1])
        #print("proxy " + vote[2])
        print("operation: " + vote[3])
        print("options: " + vote[4] )
        print("MKR: " + vote[5])
        
def removeElements(top, proposals):
    prop = proposals[0]
    while(int(prop[0]) > top):
        del proposals[0]
        prop = proposals[0]
def vectorize(string, num):
    return [string] * num
def insertDF(string, df, position):
    a = vectorize(string, df.shape[0])
    #print(df.shape[1])
    #print("position" + str(position))
    #print("string " + string)
    #print(a)
    df.insert(int(position), string, a)
    #print(df)

#define variables
filename = "makerDao.csv"
fields = ['coinName', 'ProposalId', 'proposalDate', 'proposalName', 'result', 'voteTime', 'voteAdress', 'voteOperation', 'voteOptions', 'MKR(weight)'] 
coinURL = "https://beta.mcdgov.info/"
proposalTableID = "polls-table"
voteTableID = "sorted-table"
voteUrl = "https://beta.mcdgov.info/poll/"
downloadLocation = "/home/jacob/Downloads/Operations.csv"
        
proposals = getTable(coinURL, proposalTableID, proposalTableID)
#the first two proposals are bad. 
del proposals[1]
del proposals[0]
print(proposals[1])
removeElements(14, proposals)
for proposal in proposals:
    number = proposal[0]
    realUrl = (voteUrl + number)
    downloadCSV(realUrl, "sorted-table_filter", downloadLocation)
    votes = getVotesFromCSV(downloadLocation)
    insertDF("makerDao", votes, 0)
    #basically a lazy way to put the relevantproposal info into the df. 
    for i in range(1, 5):
        insertDF(proposal[i - 1], votes, i)    
    votes.to_csv(filename, mode='a', header=False, index = False)
    os.remove(downloadLocation)
    print(number)

            


['746', '2022-02-23', 'Community Greenlight Poll - PUNK (NTFX CryptoPunk) - February 21, 2022', 'No (Defer)']


IndexError: list index out of range

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
import os
import time

def scrapeCountyHealthSite(state, year, measureNum):
  url = "https://www.countyhealthrankings.org/app/" + state + "/" + year + "/measure/factors/" + measureNum + "/data"
  tableID = "main-inner-wrapper"
  print(url)
  driver = webdriver.Chrome()
  #driver.get(url)
  driver.implicitly_wait(3) # seconds
  driver.get(ur1l)
  time.sleep(10)
  #elem = WebDriverWait(driver, 30).until(
  #EC.presence_of_element_located((By.ID, tableID)))
  soup = BeautifulSoup(driver.page_source.encode('utf-8'))
  driver.close()
  table = soup.find(id = tableID)
  table = table.findAll('tr')
  for row in table:
        cols = row.findAll("td")
        for col in cols:
            if(col.text):
                print(col.text)
        
  #print(table.prettify)

  #page = req.get(url, timeout = 5)



  #print(soup.prettify)
  #table = bs.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="Table1") 
  #table = soup.table
  #print(table)
  
scrapeCountyHealthSite("idaho", "2021" , "168")

https://www.countyhealthrankings.org/app/idaho/2021/measure/factors/168/data


NameError: name 'ur1l' is not defined